In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import sqlalchemy

import requests as requests

import httpx 
from functools import partial
from asyncio import gather, run
from aiometer import run_all

In [114]:
load_dotenv()
api_key = os.environ.get('riot_api_key')
limit_requests_per_second = 0.8 # half from api documentation

In [3]:
engine = sqlalchemy.create_engine("sqlite:///../data/database.db")

In [100]:
# funcao para leaderboard

def get_soloq_leaderboard(top=None):

    root = "https://br1.api.riotgames.com/"

    endpoints = {
        "challenger_endpoint": "lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5",
        "grandmaster_endpoint": "lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5",
        "master_endpoint": "lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5"
    }

    responses = {}  

    dataframes = {}

    for name, endpoint in endpoints.items():
        response = requests.get(f"{root}{endpoint}?api_key={api_key}")
        responses[f"{name}_response"] = response

    for name, response in responses.items():
        rank_name = name.split('_')[0]
        df = pd.DataFrame(response.json()['entries'])
        dataframes[f'{rank_name}_df'] = df

    leaderboard_df = (
        pd.concat(dataframes.values(), ignore_index=True)
        .drop(columns='rank')  
        .sort_values('leaguePoints', ascending=False)  
        .reset_index(drop=True)  
        .rename_axis('rank').reset_index()  
        .sort_values('rank', ascending=True)  
        .query('leaguePoints > 0')
        .rename(
            columns={
            'summonerId': 'summoner_id',
            'leaguePoints': 'league_points',
            'freshBlood': 'fresh_blood',
            'hotStreak': 'hot_streak',
        })
    )
    
    leaderboard_df['rank'] += 1

    return leaderboard_df

In [101]:
df_leaderboard = get_soloq_leaderboard()

# df_leaderboard.to_sql("tb_leaderboard", engine, if_exists="replace", index=False)

In [105]:
summoners = df_leaderboard['summoner_id'].to_list()

In [ ]:
# async def get_puuid(summoner_id, nr_matches):
#     async with httpx.AsyncClient(base_url='https://br1.api.riotgames.com') as client:

#         response = await client.get(f'lol/summoner/v4/summoners/{summoner_id}?api_key={api_key}')
#         print('Get puuid: ', response.status_code)
#         puuid = response.json().get('puuid')

#         response = await client.get(f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={nr_matches}&api_key={api_key}')
#         print('Get matches: ', response.status_code)
#         matches = response.json()

#         return {
#             "summoner_id": summoner_id,
#             "puuid": puuid,
#             "nr_matches": len(matches),
#             "matches": matches,
#         }
    
# async def get_match_info(match_id):
#     async with httpx.AsyncClient(base_url='https://americas.api.riotgames.com') as client:

#         response = await client.get(f'lol/match/v5/matches/{match_id}?api_key={api_key}')
#         print("Get match info", response.status_code)
#         match_info = response.json()

#         return {"match_id": match_id, **match_info}

# async def main():
#     puuid_result = await run_all(
#         [partial(get_puuid, summoner_id, 20) for summoner_id in summoners],    
#         max_per_second=limit_requests_per_second,
#         max_at_once=3  
#     )

#     df_account = pd.DataFrame(puuid_result)

#     all_match_ids = [match for matches in df_account["matches"] for match in matches]
#     match_details = await run_all(
#         [partial(get_match_info, match_id) for match_id in all_match_ids],
#         max_per_second=limit_requests_per_second,
#         max_at_once=3
#     )

#     df_match_details = pd.DataFrame(match_details)

#     return df_account, df_match_details

In [ ]:
async def get_puuid(summoner_id, nr_matches):
    async with httpx.AsyncClient(base_url='https://br1.api.riotgames.com') as client:

        response = await client.get(f'lol/summoner/v4/summoners/{summoner_id}?api_key={api_key}')
        print('Get puuid: ', response.status_code)
        puuid = response.json().get('puuid')

        response = await client.get(f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={nr_matches}&api_key={api_key}')
        print('Get matches: ', response.status_code)
        matches = response.json()

        return {
            "summoner_id": summoner_id,
            "puuid": puuid,
            "nr_matches": len(matches),
            "matches": matches,
        }
    
async def get_match_info(match_id):
    async with httpx.AsyncClient(base_url='https://americas.api.riotgames.com') as client:

        response = await client.get(f'lol/match/v5/matches/{match_id}?api_key={api_key}')
        print("Get match info", response.status_code)
        match_info = response.json()

        return {"match_id": match_id, **match_info}

async def main():
    puuid_result = await run_all(
        [partial(get_puuid, summoner_id, 20) for summoner_id in summoners],    
        max_per_second=limit_requests_per_second,
        max_at_once=3  
    )

    df_account = pd.DataFrame(puuid_result)

    return df_account

In [ ]:
df_matches_per_puuid = await main()

In [ ]:
async def get_match_info(match_id):
    async with httpx.AsyncClient(base_url='https://americas.api.riotgames.com') as client:

        response = await client.get(f'lol/match/v5/matches/{match_id}?api_key={api_key}')
        print("Get match info", response.status_code)
        match_info = response.json()

        return {"match_id": match_id, **match_info}
    
async def main():

    all_match_ids = [match for matches in df_matches_per_puuid["matches"] for match in matches]
    match_details = await run_all(
        [partial(get_match_info, match_id) for match_id in all_match_ids],
        max_per_second=limit_requests_per_second,
        max_at_once=3
    )

    df_match_details = pd.DataFrame(match_details)

    return df_match_details

In [ ]:
df_match_info = await main()